In [1]:
import csv
from collections import Counter, defaultdict

import csv
import os
from tqdm import tqdm
import pandas as pd

In [5]:
#from cda_data_cleaning.fact_extraction.measurement_extraction import MeasurementTagger
from taggers.measurement_tagger import MeasurementTagger
from taggers.measurement_token_tagger import MeasurementTokenTagger
from taggers.measurement_object_tagger.measurement_object_tagger import MeasurementObjectTagger
from estnltk import Text

In [6]:
token_tagger = MeasurementTokenTagger(output_layer = "measurement_tokens")
tagger = MeasurementTagger()

### Precision 2020

In [2]:
pwd

'/home/dage/Desktop/cda-data-cleaning/cda_data_cleaning/fact_extraction/measurement_extraction/development'

In [7]:
tp_counts = Counter()
with open("../measurements_quality/labelled/precision_to_label.csv", "r") as fin:
    reader = csv.reader(fin)
    for row in reader:
        if row[0] != 'APGAR':
            if row[-1] != 'v':
                tp_counts[row[0]] += 1

In [8]:
tp_counts.most_common()

[('KUSIHAPE', 100),
 ('LDL-KOLESTEROOL', 100),
 ('LUUTIHEDUS', 100),
 ('MCV', 100),
 ('VÖÖÜMBERMÕÕT', 100),
 ('ASAT', 100),
 ('HDL-KOLESTEROOL', 100),
 ('TRIGLÜTSERIIDID', 100),
 ('ALAT', 99),
 ('HEMOGLOBIIN', 99),
 ('KOLESTEROOL', 99),
 ('LEUKOTSÜÜDID', 98),
 ('PSA', 98),
 ('KREATINIIN', 98),
 ('KAAL', 97),
 ('GLÜKOHEMOGLOBIIN', 97),
 ('KMI', 95),
 ('GLÜKOOS', 94),
 ('PIKKUS', 93),
 ('BILIRUBIIN', 92),
 ('PULSS', 91)]

In [9]:
precisions2020 = Counter()
for k, v in tp_counts.items():
    precisions2020[k] = v/100

In [18]:
precisions2020

Counter({'BILIRUBIIN': 0.92,
         'PIKKUS': 0.93,
         'KUSIHAPE': 1.0,
         'ALAT': 0.99,
         'GLÜKOOS': 0.94,
         'HEMOGLOBIIN': 0.99,
         'KAAL': 0.97,
         'KMI': 0.95,
         'LDL-KOLESTEROOL': 1.0,
         'LEUKOTSÜÜDID': 0.98,
         'LUUTIHEDUS': 1.0,
         'MCV': 1.0,
         'PULSS': 0.91,
         'PSA': 0.98,
         'VÖÖÜMBERMÕÕT': 1.0,
         'KREATINIIN': 0.98,
         'GLÜKOHEMOGLOBIIN': 0.97,
         'KOLESTEROOL': 0.99,
         'ASAT': 1.0,
         'HDL-KOLESTEROOL': 1.0,
         'TRIGLÜTSERIIDID': 1.0})

In [10]:
df = pd.DataFrame(pd.Series(precisions2020), columns = ['new_on_new_prec'])

### Precision on 2014 data

In [11]:
path = '../measurements_quality/labelled_2014/'

In [12]:
files = os.listdir(path)
data = defaultdict(list)
for f in files:
    if f.endswith('csv'):
        fname = os.path.join(path, f)
        with open(fname, 'r', encoding = '1252') as fin:
            reader = csv.reader(fin)
            next(reader)
            for row in reader:
                if row[9] in precisions2020:
                    left = row[12].replace(row[13], '')
                    data[row[9]].append([left +' ' + row[13] + ' ' + row[14], row[15], row[16], row[30]])
                elif row[9] == 'BMI':    
                    left = row[12].replace(row[13], '')
                    data['KMI'].append([left +' ' + row[13] + ' ' + row[14], row[15], row[16], row[30]])

In [38]:
data.keys()

dict_keys(['KAAL', 'KREATINIIN', 'PIKKUS', 'TRIGLÜTSERIIDID', 'PULSS', 'PSA', 'KUSIHAPE', 'KOLESTEROOL', 'KMI', 'LDL-KOLESTEROOL', 'ALAT', 'HDL-KOLESTEROOL', 'HEMOGLOBIIN', 'GLÜKOHEMOGLOBIIN', 'ASAT', 'VÖÖÜMBERMÕÕT'])

In [37]:
len(precisions2020)

21

In [13]:
from estnltk import Text

In [14]:
tagged_texts_old = {}
tagged_texts = {}
for obj, rows in tqdm(data.items()):
    tp = []
    fp = []
    fn = []
    tn = []
    tp_old = []
    fp_old = []

    for row in rows:
        t = Text(row[0])
        token_tagger.tag(t)
        tagger.tag(t)
        if len(t.measurements) > 0:
            if row[-1] == 'n':
                fp.append((t, row[-2]))
                fp_old.append((t, row[-2]))
            else:
                tp.append((t, row[-2]))
                tp_old.append((t, row[-2]))
        else:
            if row[-1] == 'n':
                tn.append((t, row[-2]))
                fp_old.append((t, row[-2]))
            else:
                fn.append((t, row[-2]))
                tp_old.append((t, row[-2]))
    tagged_texts[obj] = [tp, fp, fn, tn] 
    tagged_texts_old[obj] = [tp_old, fp_old]

100%|██████████| 16/16 [01:47<00:00,  6.29s/it]


In [46]:
tagged_texts

{'KAAL': [[(Text(text=' Kaal 1245 g  ( 10-50P )'), '1245'),
   (Text(text='Laps on sündinud ajalisena ,  sünnikaal ~ 3 kg  , pikkus 48 cm'),
    '3'),
   (Text(text='amine ) tulemus : pikkus : 53cm  kaal : 3.644kg  KMI : 13'),
    '3.644'),
   (Text(text='lt suurtest kogustest osaliselt  Kaal 3,7kg  Iste seedimata-teeb kuurina Gef'),
    '3.7'),
   (Text(text='08.03.2012 -  Kaal 3,8 kg ;sööb Aptamili 620 ml ööpäevas'),
    '3.8'),
   (Text(text='amine ) tulemus : pikkus : 55cm  kaal : 3.84kg  KMI : 12.7'),
    '3.84'),
   (Text(text=' Sünnikaal 3,992 kg  , pikkus 54 cm'), '3.992'),
   (Text(text='1 laps - S/K/ kaal - 4,5 kg  ! ,'), '4.5'),
   (Text(text='Eile  kaal 5,470 g '), '5.47'),
   (Text(text='ine ) tulemus : pikkus : 62.5cm  kaal : 5.83kg  KMI : 14.9'),
    '5.83'),
   (Text(text='amine ) tulemus : pikkus : 64cm  kaal : 6.66kg  KMI : 16.3'),
    '6.66'),
   (Text(text='tulemus : pikkus : 70.5cm + 1cm  kaal : 7.86kg  + 230g - kuugaKMI : 15.8'),
    '7.86'),
   (Text(text='05.201

In [15]:
metrics_prec = {}
new_metrics_rec = {}
#metrics_counts = 
for el, metrics in tagged_texts.items():
    metrics_prec[el] = {}
    prec = len(metrics[0])/(len(metrics[0]) + len(metrics[1]))
    rec = len(metrics[0])/(len(metrics[0]) + len(metrics[2]))
    metrics_prec[el]['new_on_old_prec'] = prec
    new_metrics_rec[el]= rec

for el, metrics in tagged_texts_old.items():
    prec = len(metrics[0])/(len(metrics[0]) + len(metrics[1]))
    metrics_prec[el]['old_on_old_prec']= prec  

In [16]:
metrics_prec

{'KAAL': {'new_on_old_prec': 0.9774368231046932,
  'old_on_old_prec': 0.9446666666666667},
 'KREATINIIN': {'new_on_old_prec': 1.0, 'old_on_old_prec': 1.0},
 'PIKKUS': {'new_on_old_prec': 0.9953488372093023, 'old_on_old_prec': 0.963},
 'TRIGLÜTSERIIDID': {'new_on_old_prec': 1.0, 'old_on_old_prec': 1.0},
 'PULSS': {'new_on_old_prec': 0.9844559585492227, 'old_on_old_prec': 0.94},
 'PSA': {'new_on_old_prec': 0.9851116625310173, 'old_on_old_prec': 0.988},
 'KUSIHAPE': {'new_on_old_prec': 0.9970674486803519, 'old_on_old_prec': 0.986},
 'KOLESTEROOL': {'new_on_old_prec': 1.0, 'old_on_old_prec': 0.978},
 'KMI': {'new_on_old_prec': 1.0, 'old_on_old_prec': 1.0},
 'LDL-KOLESTEROOL': {'new_on_old_prec': 1.0, 'old_on_old_prec': 1.0},
 'ALAT': {'new_on_old_prec': 1.0, 'old_on_old_prec': 1.0},
 'HDL-KOLESTEROOL': {'new_on_old_prec': 1.0, 'old_on_old_prec': 1.0},
 'HEMOGLOBIIN': {'new_on_old_prec': 0.9855818743563337,
  'old_on_old_prec': 0.959},
 'GLÜKOHEMOGLOBIIN': {'new_on_old_prec': 0.994708994708

In [17]:
df2 = pd.DataFrame.from_dict(metrics_prec).T

In [18]:
df3 = pd.concat([df, df2], axis = 1).reindex(df.index)

In [19]:
df3

,new_on_new_prec,new_on_old_prec,old_on_old_prec
BILIRUBIIN,0.92,NaN,NaN
PIKKUS,0.93,0.995349,0.963000
KUSIHAPE,1.00,0.997067,0.986000
ALAT,0.99,1.000000,1.000000
GLÜKOOS,0.94,NaN,NaN
HEMOGLOBIIN,0.99,0.985582,0.959000
KAAL,0.97,0.977437,0.944667
KMI,0.95,1.000000,1.000000
LDL-KOLESTEROOL,1.00,1.000000,1.000000
LEUKOTSÜÜDID,0.98,NaN,NaN


### Recall 2020

In [20]:
path = '../measurements_quality/labelled_recall'

In [21]:
os.listdir(path)

['vöö.csv',
 'hemoglobiin.csv',
 'alat_.csv',
 'glükoos.csv',
 'kaal.csv',
 'leuko.csv',
 'psa.csv',
 'bilirubiin.csv',
 'kmi.csv',
 'ldlkol.csv',
 'pikkus.csv',
 'pulss.csv']

In [22]:
fname_to_mo = {'vöö.csv': 'VÖÖÜMBERMÕÕT', 'hemoglobiin.csv': 'HEMOGLOBIIN', 'alat_.csv': 'ALAT', 'glükoos.csv': 'GLÜKOOS', 'leuko.csv': 'LEUKOTSÜÜDID', 'psa.csv': 'PSA', 'bilirubiin.csv': 'BILIRUBIIN', 'kmi.csv': 'KMI', 'ldlkol.csv': 'LDL-KOLESTEROOL', 'pikkus.csv': 'PIKKUS', 'pulss.csv': 'PULSS', 'kaal.csv': 'KAAL'}

In [2]:
false_neg_counts = {}
for fname in os.listdir(path):
    with open(os.path.join(path, fname), 'r') as fin:
        mo = fname_to_mo[fname]
        false_neg_counts[mo] = 0
        reader = csv.reader(fin)
        for row in reader:
            if row[3] == 'v' or row[3] == '?' or row[3] == 'n':
                false_neg_counts[mo] += 1

NameError: name 'os' is not defined

In [24]:
measurement_counts = {'PULSS': 118594,
             'HEMOGLOBIIN': 72684,
             'LEUKOTSÜÜDID': 69461,
             'MCV': 53533,
             'GLÜKOOS': 45378,
             'ALAT': 28023,
             'LDL-KOLESTEROOL': 11577,
             'PSA': 12515,
             'BILIRUBIIN': 11103,
             'KAAL': 33476,
             'PIKKUS': 26696,
             'KMI': 18262,
             'VÖÖÜMBERMÕÕT': 3941}

In [25]:
token_only_counts = {'PULSS': 104579,
             'BILIRUBIIN': 38015,
             'HEMOGLOBIIN': 20184,
             'VÖÖÜMBERMÕÕT': 1081,
             'LEUKOTSÜÜDID': 31749,
             'PIKKUS': 45882,
             'GLÜKOOS': 34823,
             'ALAT': 6044,
             'KAAL': 36812,
             'MCV': 4101,
             'LDL-KOLESTEROOL': 1079,
             'PSA': 9345,
             'KMI': 2562}

In [26]:
tp_counts

Counter({'BILIRUBIIN': 92,
         'PIKKUS': 93,
         'KUSIHAPE': 100,
         'ALAT': 99,
         'GLÜKOOS': 94,
         'HEMOGLOBIIN': 99,
         'KAAL': 97,
         'KMI': 95,
         'LDL-KOLESTEROOL': 100,
         'LEUKOTSÜÜDID': 98,
         'LUUTIHEDUS': 100,
         'MCV': 100,
         'PULSS': 91,
         'PSA': 98,
         'VÖÖÜMBERMÕÕT': 100,
         'KREATINIIN': 98,
         'GLÜKOHEMOGLOBIIN': 97,
         'KOLESTEROOL': 99,
         'ASAT': 100,
         'HDL-KOLESTEROOL': 100,
         'TRIGLÜTSERIIDID': 100})

In [1]:
false_neg_counts

NameError: name 'false_neg_counts' is not defined

In [28]:
measurement_to_token_ratio = {}
for k, v in measurement_counts.items():
    ratio = v/token_only_counts[k]
    measurement_to_token_ratio[k] = ratio

In [29]:
measurement_to_token_ratio

{'PULSS': 1.1340135208789528,
 'HEMOGLOBIIN': 3.6010701545778834,
 'LEUKOTSÜÜDID': 2.1878169391161926,
 'MCV': 13.0536454523287,
 'GLÜKOOS': 1.3031042701662694,
 'ALAT': 4.636499007279947,
 'LDL-KOLESTEROOL': 10.72937905468026,
 'PSA': 1.3392188336008561,
 'BILIRUBIIN': 0.2920689201630935,
 'KAAL': 0.9093773769423015,
 'PIKKUS': 0.5818403731310754,
 'KMI': 7.128024980483997,
 'VÖÖÜMBERMÕÕT': 3.6456984273820536}

In [30]:
recalls = {}
for k, v in false_neg_counts.items():
    fn = v
    tp = tp_counts[k]*measurement_to_token_ratio[k]
    recall = tp/(tp+fn)
    recalls[k] = recall

In [31]:
recalls

{'VÖÖÜMBERMÕÕT': 0.9358266547937073,
 'HEMOGLOBIIN': 0.9295969193420116,
 'ALAT': 0.9622652111343849,
 'GLÜKOOS': 0.7227004515086121,
 'KAAL': 0.6671951344798537,
 'LEUKOTSÜÜDID': 0.9640297571082412,
 'PSA': 0.8454041013268999,
 'BILIRUBIIN': 0.6267815987570841,
 'KMI': 0.8908128014064952,
 'LDL-KOLESTEROOL': 0.9218171351130554,
 'PIKKUS': 0.7401217354909218,
 'PULSS': 0.7010772707754491}

In [32]:
recalls_2014 = {'PULSS': 0.77,
             #'BILIRUBIIN': 38015,
             'HEMOGLOBIIN': 0.96,
             'VÖÖÜMBERMÕÕT': 0.36,
             #'LEUKOTSÜÜDID': 31749,
             'PIKKUS': 0.81,
             'GLÜKOOS': 0.57,
             'ALAT': 0.96,
             'KAAL': 0.75,
             #'MCV': 4101,
             'LDL-KOLESTEROOL': 0.88,
                'HDL-KOLESTEROOL': 0.83,
             'PSA': 0.96,
               'KREATINIIN': 0.93,
               'GLÜKOHEMOGLOBIIN': 0.87,
               'KOLESTEROOL': 0.9,
               'KMI': 0.66,
               'ASAT': 0.97,
               'KUSIHAPE': 0.95,
               'TRIGLÜTSERIIDID': 0.82}

In [33]:
recalls_2014

{'PULSS': 0.77,
 'HEMOGLOBIIN': 0.96,
 'VÖÖÜMBERMÕÕT': 0.36,
 'PIKKUS': 0.81,
 'GLÜKOOS': 0.57,
 'ALAT': 0.96,
 'KAAL': 0.75,
 'LDL-KOLESTEROOL': 0.88,
 'HDL-KOLESTEROOL': 0.83,
 'PSA': 0.96,
 'KREATINIIN': 0.93,
 'GLÜKOHEMOGLOBIIN': 0.87,
 'KOLESTEROOL': 0.9,
 'KMI': 0.66,
 'ASAT': 0.97,
 'KUSIHAPE': 0.95,
 'TRIGLÜTSERIIDID': 0.82}

In [34]:
new_metrics_rec

{'KAAL': 0.7642907551164432,
 'KREATINIIN': 0.696,
 'PIKKUS': 0.8888888888888888,
 'TRIGLÜTSERIIDID': 0.756,
 'PULSS': 0.6063829787234043,
 'PSA': 0.8036437246963563,
 'KUSIHAPE': 0.6896551724137931,
 'KOLESTEROOL': 0.8650306748466258,
 'KMI': 0.912,
 'LDL-KOLESTEROOL': 0.532,
 'ALAT': 0.63,
 'HDL-KOLESTEROOL': 0.53,
 'HEMOGLOBIIN': 0.9979144942648592,
 'GLÜKOHEMOGLOBIIN': 0.7550200803212851,
 'ASAT': 0.652,
 'VÖÖÜMBERMÕÕT': 0.9898373983739838}

In [35]:
df3['new_on_new_rec'] = pd.Series(recalls)

In [36]:
df3['old_on_old_rec'] = pd.Series(recalls_2014)

In [37]:
df3['new_on_old_rec'] = pd.Series(new_metrics_rec)

In [38]:
df3

,new_on_new_prec,new_on_old_prec,old_on_old_prec,new_on_new_rec,old_on_old_rec,new_on_old_rec
BILIRUBIIN,0.92,NaN,NaN,0.626782,NaN,NaN
PIKKUS,0.93,0.995349,0.963000,0.740122,0.81,0.888889
KUSIHAPE,1.00,0.997067,0.986000,NaN,0.95,0.689655
ALAT,0.99,1.000000,1.000000,0.962265,0.96,0.630000
GLÜKOOS,0.94,NaN,NaN,0.722700,0.57,NaN
HEMOGLOBIIN,0.99,0.985582,0.959000,0.929597,0.96,0.997914
KAAL,0.97,0.977437,0.944667,0.667195,0.75,0.764291
KMI,0.95,1.000000,1.000000,0.890813,0.66,0.912000
LDL-KOLESTEROOL,1.00,1.000000,1.000000,0.921817,0.88,0.532000
LEUKOTSÜÜDID,0.98,NaN,NaN,0.964030,NaN,NaN


In [39]:
df3['f1_2020'] = 2*(df3['new_on_new_prec']*df3['new_on_new_rec'])/(df3['new_on_new_prec']+df3['new_on_new_rec'])

In [40]:
df3

,new_on_new_prec,new_on_old_prec,old_on_old_prec,new_on_new_rec,old_on_old_rec,new_on_old_rec,f1_2020
BILIRUBIIN,0.92,NaN,NaN,0.626782,NaN,NaN,0.745599
PIKKUS,0.93,0.995349,0.963000,0.740122,0.81,0.888889,0.824267
KUSIHAPE,1.00,0.997067,0.986000,NaN,0.95,0.689655,NaN
ALAT,0.99,1.000000,1.000000,0.962265,0.96,0.630000,0.975936
GLÜKOOS,0.94,NaN,NaN,0.722700,0.57,NaN,0.817151
HEMOGLOBIIN,0.99,0.985582,0.959000,0.929597,0.96,0.997914,0.958848
KAAL,0.97,0.977437,0.944667,0.667195,0.75,0.764291,0.790595
KMI,0.95,1.000000,1.000000,0.890813,0.66,0.912000,0.919455
LDL-KOLESTEROOL,1.00,1.000000,1.000000,0.921817,0.88,0.532000,0.959318
LEUKOTSÜÜDID,0.98,NaN,NaN,0.964030,NaN,NaN,0.971949


In [41]:
df3['f1_2014'] = 2*(df3['old_on_old_prec']*df3['old_on_old_rec'])/(df3['old_on_old_prec']+df3['old_on_old_rec'])

In [42]:
df3.sort_values(by=['f1_2020'], ascending = False)

,new_on_new_prec,new_on_old_prec,old_on_old_prec,new_on_new_rec,old_on_old_rec,new_on_old_rec,f1_2020,f1_2014
ALAT,0.99,1.000000,1.000000,0.962265,0.96,0.630000,0.975936,0.979592
LEUKOTSÜÜDID,0.98,NaN,NaN,0.964030,NaN,NaN,0.971949,NaN
VÖÖÜMBERMÕÕT,1.00,0.983838,0.984000,0.935827,0.36,0.989837,0.966850,0.527143
LDL-KOLESTEROOL,1.00,1.000000,1.000000,0.921817,0.88,0.532000,0.959318,0.936170
HEMOGLOBIIN,0.99,0.985582,0.959000,0.929597,0.96,0.997914,0.958848,0.959500
KMI,0.95,1.000000,1.000000,0.890813,0.66,0.912000,0.919455,0.795181
PSA,0.98,0.985112,0.988000,0.845404,0.96,0.803644,0.907740,0.973799
PIKKUS,0.93,0.995349,0.963000,0.740122,0.81,0.888889,0.824267,0.879898
GLÜKOOS,0.94,NaN,NaN,0.722700,0.57,NaN,0.817151,NaN
PULSS,0.91,0.984456,0.940000,0.701077,0.77,0.606383,0.791992,0.846550
